# **Procesamiento de Lenguaje Natural**

## Maestría en Inteligencia Artificial Aplicada
#### Tecnológico de Monterrey
#### Prof Luis Eduardo Falcón Morales

### **Adtividad en Equipos: sistema LLM + RAG**

* **Nombres y matrículas:**

  * Fernando Omar Salazar Ortiz - A01796214
  * Carlos Aaron Bocanegra Buitron - A01796345
  * Luis Enrique González González - A01795338
  * Gloria María Campos García - A01422345

* **Número de Equipo: 12**


* ##### **El formato de este cuaderno de Jupyter es libre, pero debe incuir al menos las siguientes secciones:**

  * ##### **Introducción de la problemática a resolver.**
  * ##### **Sistema RAG + LLM**
  * ##### **El chatbot, incluyendo ejemplos de prueba.**
  * ##### **Conclusiones**

* ##### **Pueden importar los paquetes o librerías que requieran.**

* ##### **Pueden incluir las celdas y líneas de código que deseen.**

## 1. Introducción

El código desarrollado implementa un sistema de recuperación de información basado en **RAG** para responder preguntas sobre alimentación durante el embarazo, utilizando como fuente una guía alimentaria para mujeres embarazadas en formato PDF. Se extrae, procesa y almacena el contenido relevante, filtrando elementos complejos como tablas y ciertas secciones (pies de página) que no añaden valor. Posteriormente, se generan embeddings de los fragmentos de texto y se utiliza FAISS para realizar búsquedas semánticas rápidas. Al recibir una pregunta, el sistema recupera los fragmentos más relevantes y los emplea como contexto para generar respuestas precisas a través de un modelo de lenguaje de OpenAI.


- **Retrieval-Augmented Generation (RAG)**
es un enfoque que combina la recuperación de información con modelos generativos de lenguaje para mejorar la precisión y relevancia de las respuestas generadas. En lugar de depender únicamente del conocimiento preentrenado de un modelo, RAG recupera fragmentos de texto relevantes de una fuente externa y los utiliza como contexto para generar respuestas más fundamentadas. Este método ha demostrado ser efectivo para reducir la generación de información errónea y mejorar la coherencia en tareas de procesamiento de lenguaje natural (Aytar et al., 2024).
------
- **Large Language Models (LLM)**
son modelos de inteligencia artificial entrenados con grandes volúmenes de datos textuales para comprender y generar lenguaje natural de manera sofisticada. Estos modelos utilizan arquitecturas avanzadas de redes neuronales, como los transformadores, para procesar texto con alta precisión y generar respuestas contextualmente relevantes. Su aplicación abarca desde asistentes virtuales hasta generación de contenido y traducción automática (Naveed et al., 2024).
------
- **FAISS (Facebook AI Similarity Search)**
es una biblioteca optimizada para la búsqueda eficiente de similitud entre vectores de alta dimensión. Se utiliza principalmente en aplicaciones de recuperación de información y aprendizaje automático para encontrar los elementos más similares dentro de grandes conjuntos de datos embebidos. FAISS emplea estructuras de índices avanzadas y técnicas de cuantización para acelerar las búsquedas, permitiendo realizar consultas en tiempo real con grandes volúmenes de datos (Douze et al., 2025).


## 2. Librerías

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
google_drive_path_parent = "/content/drive/MyDrive/Colab Notebooks/NLP/Actividades/ActividadSemana9_Chatbot_LLM_RAG/"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#Se hacen las instalaciones necesarias
!pip install -q openai pymupdf faiss-cpu

In [ ]:
#Se hacen las importaciones necesarias
from google.colab import userdata
import openai
import fitz #PyMuPDF
import tiktoken
import numpy as np
import faiss
import pickle
import json
import re


## 3. Open AI

La API de OpenAI permite acceder a modelos avanzados de inteligencia artificial para realizar tareas como generación de texto, análisis semántico y generación de embeddings. Aqui se configura la autenticación mediante una API Key almacenada en los secretos del usuario, garantizando el acceso seguro a los servicios de OpenAI.

In [ ]:
# Se carga la API Key de OpenAI
api_key = userdata.get("OPENAI_API_KEY_PERSONAL")

if not api_key:
  raise ValueError("API key no encontrada en los secretos")

client = openai.OpenAI(api_key=api_key)

## 4. Extracción y procesamiento de texto desde PDF

En esta sección, se implementa un sistema de extracción y procesamiento de texto desde un documento PDF de una guía alimentaria para mujeres embarazadas.
El proceso comienza con la lectura del archivo PDF y la segmentación del contenido en fragmentos de texto significativos, eliminando encabezados, pies de página y elementos complejos como tablas. Posteriormente, los fragmentos filtrados se almacenan en formato JSON, lo que permite su edición y estructuración para una recuperación eficiente.
Esta etapa es clave para asegurar que la información proporcionada al usuario sea precisa, relevante y fundamentada en fuentes confiables.

In [ ]:
def extract_text_from_pdf(
    pdf_path,
    use_blocks=True,
    min_length=40,
    exclude_headers_footers=True,
    header_height=60,
    footer_height=60,
    skip_pages=None,
    start_at_page=0,
    stop_at_page=None,
    omit_texts=None
):
    """
    Extrae texto de un PDF usando PyMuPDF, ordenando bloques visuales y controlando
    rango de páginas y omisión de textos específicos.

    Args:
        pdf_path (str): Ruta del archivo PDF.
        use_blocks (bool): Si True, extrae texto por bloques visuales; si False, por página completa.
        min_length (int): Longitud mínima para conservar un bloque.
        exclude_headers_footers (bool): Si True, excluye encabezado y pie de página.
        header_height (int): Altura desde arriba a excluir (en puntos).
        footer_height (int): Altura desde abajo a excluir (en puntos).
        skip_pages (List[int] or None): Índices de páginas (base 0) a omitir.
        start_at_page (int): Página inicial (índice base 0) para empezar a procesar.
        stop_at_page (int or None): Índice de página en la que se debe detener el procesamiento.
        omit_texts (List[str] or None): Lista de textos exactos a omitir (case-insensitive, strip).

    Returns:
        List[str]: Lista de chunks de texto extraídos del PDF.
    """
    doc = fitz.open(pdf_path)
    all_chunks = []
    skip_pages = skip_pages or []
    omit_texts = [t.strip().lower() for t in (omit_texts or [])]

    start_marker = "lunes martes miércoles jueves viernes sábado domingo"
    end_marker = "oriente sobre cómo poner en práctica la recomendación, mencionando que:"
    skipping = False

    # Itera desde start_at_page hasta stop_at_page (no inclusivo)
    page_range = range(start_at_page, stop_at_page if stop_at_page is not None else len(doc))

    for i in page_range:
        if i in skip_pages:
            continue

        page = doc[i]
        page_height = page.rect.height

        if use_blocks:
            blocks = page.get_text("blocks")
            blocks = sorted(blocks, key=lambda b: (b[1], b[0]))
            for b in blocks:
                x0, y0, x1, y1, text = b[:5]
                if not text or len(text.strip()) < min_length:
                    continue
                if exclude_headers_footers and (y0 < header_height or y1 > page_height - footer_height):
                    continue
                clean_text = re.sub(r'\s+', ' ', text.strip().replace("\n", " ").replace("\t", " "))
                lower_clean = clean_text.lower()
                if lower_clean in omit_texts:
                    continue
                if start_marker in lower_clean:
                    skipping = True
                    continue
                if end_marker in lower_clean:
                    skipping = False
                    continue
                if skipping:
                    continue
                all_chunks.append(clean_text)
        else:
            text = page.get_text()
            if text and len(text.strip()) >= min_length:
                clean_text = re.sub(r'\s+', ' ', text.strip().replace("\n", " ").replace("\t", " "))
                lower_clean = clean_text.lower()
                if lower_clean in omit_texts:
                    continue
                if start_marker in lower_clean:
                    skipping = True
                    continue
                if end_marker in lower_clean:
                    skipping = False
                    continue
                if skipping:
                    continue
                all_chunks.append(clean_text)

    return all_chunks



def save_chunks_to_json(chunks, file_path="chunks.json"):
    """
    Guarda una lista de bloques de texto en un archivo JSON.
    Cada línea es un bloque, editable manualmente.
    """
    with open(file_path, "w", encoding="utf-8") as f:
        json.dump(chunks, f, ensure_ascii=False, indent=2)
    print(f"✅ Bloques guardados en {file_path}")


def load_chunks_from_json(file_path="chunks.json"):
    """
    Carga los bloques de texto desde un archivo JSON y devuelve una lista.
    """
    with open(file_path, "r", encoding="utf-8") as f:
        chunks = json.load(f)
    print(f"📄 Bloques cargados: {len(chunks)}")
    return chunks

## 5. Generación y gestión de embeddings

En esta sección se generan los embeddings, estos se crean utilizando el modelo **text-embedding-3-small** de OpenAI, permitiendo que los fragmentos de texto sean codificados en un espacio vectorial donde términos con significados similares se encuentran cercanos. Posteriormente, estos embeddings se almacenan en archivos .pkl para facilitar su reutilización y recuperación eficiente.
Este proceso es fundamental para realizar búsquedas semánticas y seleccionar la información más relevante en función de las preguntas del usuario.

In [ ]:
def embed_texts_openai(texts):
    embeddings = []
    for chunk in texts:
        response = client.embeddings.create(
            input=chunk,
            model="text-embedding-3-small"
        )
        embedding = response.data[0].embedding
        embeddings.append(embedding)
    return np.array(embeddings).astype("float32")

def save_embeddings_to_pkl(vectors, chunks, file_path="embeddings_data.pkl"):
    """
    Guarda los vectores (embeddings) y sus textos (chunks) en un archivo .pkl.

    Args:
        vectors (List[List[float]] o np.ndarray): Embeddings generados.
        chunks (List[str]): Lista de textos originales de cada embedding.
        file_path (str): Ruta de archivo donde se guardará todo.
    """
    data = {
        "vectors": np.array(vectors).astype("float64"),
        "chunks": chunks
    }
    with open(file_path, "wb") as f:
        pickle.dump(data, f)
    print(f"✅ Embeddings guardados en {file_path}")


def load_embeddings_from_pkl(file_path="embeddings_data.pkl"):
    """
    Carga los vectores (embeddings) y los textos (chunks) desde un archivo .pkl.

    Returns:
        Tuple[np.ndarray, List[str]]: embeddings y chunks.
    """
    with open(file_path, "rb") as f:
        data = pickle.load(f)
    print(f"📂 Cargados {len(data['chunks'])} embeddings desde {file_path}")
    return data["vectors"], data["chunks"]

## 6. Optimización de búsqueda y recuperación de información

En esta sección, se emplea **FAISS** para indexar los embeddings generados y acelerar la búsqueda de fragmentos de texto con mayor similitud con la consulta del usuario.
El índice FAISS permite realizar búsquedas rápidas en grandes volúmenes de datos embebidos, identificando los fragmentos más relevantes sin necesidad de recorrer todo el conjunto de datos. Esto es fundamental para responder preguntas con base en la información extraída del PDF de la guía alimentaria para mujeres embarazadas, asegurando respuestas fundamentadas y contextualizadas.
Además de crea la función summarize_history que va a usar un LLM para ir generando y guardando un resumen de la charla actual.

In [ ]:
def create_faiss_index(embeddings):
    dimension = len(embeddings[0])
    index = faiss.IndexFlatL2(dimension)
    index.add(embeddings)
    return index

In [ ]:
def retrieve_and_answer_openai(
    question,
    chunks,
    index,
    chunk_embeddings,
    client,
    summary=None,
    top_k=5,
    use_rag=True,
    print_chunks=False
):
    if use_rag:
        q_embedding = client.embeddings.create(
            input=question,
            model="text-embedding-3-small"
        ).data[0].embedding
        q_vector = np.array(q_embedding).astype("float32").reshape(1, -1)
        D, I = index.search(q_vector, top_k)
        if print_chunks:
            print("------------------------------------")
            for rank, idx in enumerate(I[0]):
                print(f"--- Chunk #{idx} --")
                print(f"Distancia (D): {D[0][rank]} \n")
                print(chunks[idx])
            print("------------------------------------")
        context = "\n---\n".join([chunks[i] for i in I[0]])

        if summary:
            prompt = f"""Resumen de la conversación previa:
{summary}

Usa la siguiente información para responder la pregunta de forma clara y completa. Solo responde con base en la información proporcionada, no busques datos en otras fuentes.
Información:
{context}

Pregunta: {question}
Respuesta:"""
        else:
            prompt = f"""Usa la siguiente información para responder la pregunta de forma clara y completa. Solo responde con base en la información proporcionada, no busques datos en otras fuentes.
Información:
{context}

Pregunta: {question}
Respuesta:"""
    else:
        if summary:
            prompt = f"""Resumen de la conversación previa:
{summary}

Responde de forma clara y completa la siguiente pregunta sobre alimentación durante el embarazo. Si no tienes información suficiente, dilo claramente.

Pregunta: {question}
Respuesta:"""
        else:
            prompt = f"""Responde de forma clara y completa la siguiente pregunta sobre alimentación durante el embarazo. Si no tienes información suficiente, dilo claramente.

Pregunta: {question}
Respuesta:"""

    response = client.chat.completions.create(
        model="o4-mini",
        messages=[
            {
                "role": "system",
                "content": (
                    "Eres un chatbot para resolver dudas de la alimentación durante el embarazo "
                    "que responde en base a la información proporcionada de contexto. "
                    "Si la información no es suficiente, entonces dices que no puedes responder esa pregunta."
                    "Si algún fragmento de la información proporcionada no está relacionada con la pregunta, ignóralo al hacer la respuesta."
                )
            },
            {"role": "user", "content": prompt}
        ]
    )
    return response.choices[0].message.content.strip()


In [ ]:
def summarize_history(history, client, max_turns=2):
    """
    Resume la conversación contenida en 'history'.
    Si hay menos de max_turns, devuelve el historial como string.
    """
    if len(history) < max_turns * 2:  # Cada turno son 2 entradas
        # Historial corto, solo lo concatenamos
        return "\n".join(f"{msg['role'].capitalize()}: {msg['content']}" for msg in history)

    # Si es largo, lo resumimos
    history_text = "\n".join(f"{msg['role'].capitalize()}: {msg['content']}" for msg in history)
    resumen_prompt = (
        f"Resume la siguiente conversación entre usuario y chatbot sobre alimentación durante el embarazo, "
        f"conservando los puntos clave y detalles importantes:\n\n{history_text}\n\nResumen:"
    )
    response = client.chat.completions.create(
        model="o4-mini",
        messages=[
            {"role": "system", "content": "Eres un asistente experto en resumir conversaciones de nutrición para embarazadas."},
            {"role": "user", "content": resumen_prompt}
        ]
    )
    return response.choices[0].message.content.strip()

## 7. Procesamiento

En esta sección, se preparan los datos extraídos de la guía alimentaria para mujeres embarazadas, asegurando que sean filtrados y estructurados adecuadamente para su uso en el sistema de recuperación de información.
Primero se comienza con la extracción de texto desde el archivo PDF, donde se segmentan los fragmentos en bloques, eliminando encabezados, pies de página y secciones con tablas y se establecen parámetros clave como el rango de páginas a analizar, garantizando que solo la información útil sea capturada.
Una vez extraídos los fragmentos, estos se almacenan en un archivo JSON, facilitando su edición manual para mejorar su calidad. Dejando solo el contenido textual relevante. Posteriormente, se generan y guardan embeddings de estos fragmentos, lo que permite convertir el texto en representaciones numéricas para realizar búsquedas semánticas más precisas.
Finalmente, se crea un índice FAISS, permitiendo que el sistema busque los fragmentos más relevantes al recibir una pregunta.
Este procesamiento es esencial para asegurar que la información utilizada por el modelo de lenguaje sea clara, estructurada y altamente relevante, optimizando la precisión y calidad de las respuestas generadas.


In [ ]:
# PROCESAMIENTO
'''
pdf_path = google_drive_path_parent + "guia-alimentaria-mujeres-embarazadas.pdf"
chunks_v2 = extract_text_from_pdf(
    pdf_path=pdf_path,
    use_blocks=True,
    min_length=3,
    exclude_headers_footers=True,
    header_height=0,
    footer_height=50,
    skip_pages=[],
    start_at_page=6,
    stop_at_page=78,
    omit_texts=["¿CÓMO ESTAMOS EN MÉXICO?",
                "¿POR QUÉ ES IMPORTANTE ESTA RECOMENDACIÓN?",
                "En la salud:",
                "En las mujeres:",
                "En las y los bebés:",
                "En la cultura y en la sociedad:",
                "En el ambiente:",
                "En la economía, en la cultura y en la sociedad:",
                "En el o la bebé gestante",
                "En mujeres embarazadas:",
                "En mujeres en periodo de lactancia y posparto:",
                "En las y los recién nacidos:",
                #"ORIENTE SOBRE CÓMO PONER EN PRÁCTICA LA RECOMENDACIÓN, MENCIONANDO QUE:",
                ]
)
'''

'\npdf_path = google_drive_path_parent + "guia-alimentaria-mujeres-embarazadas.pdf"\nchunks_v2 = extract_text_from_pdf(\n    pdf_path=pdf_path,\n    use_blocks=True,\n    min_length=3,\n    exclude_headers_footers=True,\n    header_height=0,\n    footer_height=50,\n    skip_pages=[],\n    start_at_page=6,\n    stop_at_page=78,\n    omit_texts=["¿CÓMO ESTAMOS EN MÉXICO?",\n                "¿POR QUÉ ES IMPORTANTE ESTA RECOMENDACIÓN?",\n                "En la salud:",\n                "En las mujeres:",\n                "En las y los bebés:",\n                "En la cultura y en la sociedad:",\n                "En el ambiente:",\n                "En la economía, en la cultura y en la sociedad:",\n                "En el o la bebé gestante",\n                "En mujeres embarazadas:",\n                "En mujeres en periodo de lactancia y posparto:",\n                "En las y los recién nacidos:",\n                #"ORIENTE SOBRE CÓMO PONER EN PRÁCTICA LA RECOMENDACIÓN, MENCIONANDO QUE:",\

In [ ]:
#save_chunks_to_json(chunks=chunks_v2, file_path= google_drive_path_parent + "chunks_v2.json")

In [ ]:
#chunks2 =load_chunks_from_json(file_path=google_drive_path_parent + "chunks_cambios_manuales.json")
#Los cambios manuales que hicieron fueron eliminar la información correspondiente a las tablas

In [ ]:
#%%time
#Se generan los embeddings
#embeddings = embed_texts_openai(chunks_v2)

In [ ]:
#save_embeddings_to_pkl(embeddings, chunks_v2, file_path=google_drive_path_parent + "embeddings_guia_alimentaria_v2.pkl")

In [ ]:
embeddings, chunks_v2 =load_embeddings_from_pkl(file_path=google_drive_path_parent + "embeddings_guia_alimentaria_v2.pkl")

📂 Cargados 287 embeddings desde /content/drive/MyDrive/Colab Notebooks/NLP/Actividades/ActividadSemana9_Chatbot_LLM_RAG/embeddings_guia_alimentaria_v2.pkl


In [ ]:
#Se crea el indice
faiss_index = create_faiss_index(embeddings)

In [ ]:

#pregunta = "¿Se puede consumir alcohol durante el embarazo?"
#pregunta = "¿Qué alimentos y bebidas se deben evitar durante el embarazo?"
#pregunta = "¿cuántas leguminosas se recomienda consumir durante el embarazo?"
#pregunta = "¿Qué frutas puedo comer durante el embarazo?" # No tiene buenos resultados esta pregunta
#pregunta = "¿Qué cereales debo comer durante el embarazo?"
#pregunta = "¿Puedo comer elote?"


#pregunta = "¿Puedo comer frutos rojos, como fresas, zarzamoras, arandanos, etc"
#pregunta = "¿Puedo comer chorizo?"
#pregunta = "¿Puedo comer carne de pescado?"
#pregunta = "¿Qué me dijsite de los frutos rojos?"
#pregunta = "¿Se puede consumir café durante el embarazo?"
#pregunta = "¿Cuánta agua se debe consumir durante el embarazo?"
#respuesta = retrieve_and_answer_openai(pregunta, chunks_v2, faiss_index, embeddings, use_rag=True, print_chunks=True)
#print("\n🤖 Respuesta del chatbot:\n", respuesta)

## 8. Aplicación en nutrición materna

En esta sección, el sistema responde preguntas sobre alimentación durante el embarazo basándose en la información previamente extraída de la guía alimentaria. Se implementa un flujo de procesamiento que recupera fragmentos de texto relevantes y los utiliza como contexto para generar respuestas fundamentadas con un modelo de lenguaje de OpenAI.
Utilizando búsqueda semántica con FAISS y embeddings generados con OpenAI, el sistema identifica los fragmentos de texto más relevantes en la guía alimentaria y los emplea en la construcción de respuestas.


In [ ]:
history = []       # Historial de turnos pregunta/resp
summary = None     # El resumen actual

while True:
    pregunta = input("Pregunta: ")
    if pregunta.lower() in ["salir"]:
        break

    # Genera respuesta con memoria-resumen
    respuesta = retrieve_and_answer_openai(
        question=pregunta,
        chunks=chunks_v2,
        index=faiss_index,
        chunk_embeddings=embeddings,
        client=client,
        summary=summary,
        use_rag=True,
        print_chunks=True
    )
    print("Chatbot:", respuesta)
    print("##################################")
    history.append({"role": "user", "content": pregunta})
    history.append({"role": "assistant", "content": respuesta})

    # Actualiza el resumen si el historial es largo
    summary = summarize_history(history, client)


Pregunta: ¿Puedo comer frutos rojos, como fresas, zarzamoras, arandanos, etc
------------------------------------
--- Chunk #65 --
Distancia (D): 0.9038963317871094 

Pruebe congelar tus frutas favoritas, ya sea plátano, fresas o uvas, entre otras.
--- Chunk #64 --
Distancia (D): 0.9370267391204834 

Incluya frutas de temporada, tanto en el desayuno, los refrigerios o en cualquier momento del día en el que tenga hambre o antojo de algo dulce.
--- Chunk #67 --
Distancia (D): 0.9628674983978271 

Incluya frutas en todas sus comidas; puede hacer combinaciones con otros grupos de alimentos, por ejemplo: cereal con fruta (avena con manzana), leche y fruta (licuado de plátano), verduras y frutas (ensalada con espinaca, lechuga y fresa o naranja en gajos), entre otras.
--- Chunk #7 --
Distancia (D): 0.9753925204277039 

· Verduras: como acelgas, brócoli, calabaza, cebolla, chile, espinaca, hongos, jitomate, lechuga, pepino, nopal y zanahoria. · Frutas: como durazno, fresa, guayaba, mango, man

# **Conclusiones:**

La combinación de Large Language Models (LLM) y Retrieval-Augmented Generation (RAG) ofrece una solución poderosa para mejorar la precisión y relevancia de las respuestas generadas por modelos de lenguaje. Mientras que los LLM son capaces de generar texto fluido y coherente, su conocimiento está limitado a los datos con los que fueron entrenados. Aquí es donde RAG entra en juego ya que al integrar un mecanismo de recuperación de información, los modelos pueden acceder a datos actualizados y específicos, reduciendo errores y mejorando la contextualización de las respuestas.

En este caso, el uso de LLM + RAG permite responder consultas nutricionales durante el embarazo con información fundamentada en una guía especializada. Al integrar procesamiento de texto, búsqueda semántica y generación automática de respuestas, se logra una solución efectiva que proporciona respuestas claras y contextualizadas. Por otro lado, la eliminación de contenido irrelevante mejora significativamente la precisión de las respuestas. En general, el modelo facilita la difusión de información confiable sobre nutrición materna, contribuyendo a la identificación de mejores hábitos alimenticios durante el embarazo.

Finalmente, la generación de un resumen del chat actual por medio de un LLM es una técnica poderosa para ahorrar tokens, disminuir el tiempo de respuesta a la vez que se genera una mejor experiencia al usuario.

Como una fase siguiente de este chatbot pensamos que podríamos denormalizar las tablas contenidas en el pdf y generar los chunks que contengan esa información para posteriormente agregarlos al índice y de esta forma el chatbot pueda responder preguntas como: ¿Cuántos gramos de carne debo de comer si tengo 8 meses de embarazo? o ¿De acuerdo a mi índice masa corporal (23), ¿cuántas porciones de pollo debo comer si estoy en el primer trimestre del embarazo?. E inclusive se podría llegar a evaluar qué tan bueno es el chatbot realizando recomendaciones de porciones de alimentos para crear una dieta semanal. Sin duda el chatbot puede ser una herramienta muy útil para las mujeres mexicanas que tienen muchas preguntas acerca de sus hábitos alimenticios para darle la mejor nutrición a su bebé.

## 10. Bibliografía

Aytar, A. Y., Kaya, K., & Kilic, K. (2024). A retrieval-augmented generation framework for academic literature navigation in data science. arXiv. https://arxiv.org/abs/2412.15404v1

Naveed, H., Khan, A. U., Qiu, S., Saqib, M., Anwar, S., Usman, M., Akhtar, N., Barnes, N., & Mian, A. (2024). A comprehensive overview of large language models. arXiv. https://arxiv.org/abs/2307.06435

Douze, M., Guzhva, A., Deng, C., Johnson, J., Szilvasy, G., Mazaré, P.-E., Lomeli, M., Hosseini, L., & Jégou, H. (2025). The Faiss library. arXiv. https://arxiv.org/abs/2401.08281

Secretaría de Salud, Instituto Nacional de Salud Pública, UNICEF. Bonvecchio-Arenas, A., Ramírez-Silva, C. I., Fernández-Gaxiola, A. C., Ayala-Niochet, M. C., & Unar-Munguía, M. (2024). Recomendaciones para la promoción de la salud en mujeres embarazadas y en periodo de lactancia. México. Recuperado de https://pronamsalud.csg.gob.mx/guia-alimentaria-mujeres-embarazadas.pdf


# **Fin de la actividad chatbot: LLM + RAG**